In [ ]:
!pip3 install face_recognition
import face_recognition
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
import cv2
import pathlib

In [ ]:
def extractImages(fileName, directory, output_directory, frame_every=1):
    print(fileName)
    pathOut = output_directory + "/" + fileName[0:-4] + "/"
    if not os.path.exists(pathOut):
        os.makedirs(pathOut)
    print(pathOut)
    count = 0
    frame_count = 1
    vidcap = cv2.VideoCapture(directory + "/" + fileName)
    success,image = vidcap.read()
    success = True
    while success:
        # vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line
        success,image = vidcap.read()
        frame_count += 1

        if success and (frame_count % frame_every == 0):
            
#             print ('Read a new frame: ', success)
            frameName = pathOut + "frame%04d.jpg" % count
            if count % 500 == 0:
                print(frameName)

            cv2.imwrite(frameName, image)     # save frame as JPEG file
            count += 1
        
        

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import PIL
from tqdm import tqdm

def load_images(imageDir):
  imgs = []
  for filename in os.listdir(imageDir):
    if filename.endswith(".jpg"):
      imgs.append((filename, np.array(PIL.Image.open(imageDir + filename))))
  return imgs

def num_images(imageDir):
  count = 0
  for filename in os.listdir(imageDir):
    if filename.endswith(".jpg"):
      count += 1
  return count

"""
    Parameters: 
        - imageDir: the file path for the image directory extract the faces from
        - face_encoder: the face encoder we want to compare to see if we have extracted the correct face
"""
def singleFaceExtracterWithExtraPixels(imageDir, directory, face_encoder, padding=0, mode='train', batch_size=128):
    
    pathOut = directory +'/'+imageDir[:-1]+'-face-crop/'
    if not os.path.exists(pathOut):
        os.makedirs(pathOut)
    print(pathOut)
    count = 0

    if mode == 'test':
      f = open(pathOut + "frame_list.txt", "w")
      f.write("frame_number, top, right, bottom, left, height, width\n")
      f.close()

    # print('here')
    # lst = load_images(directory +'/'+imageDir)
    length = num_images(directory +'/'+imageDir)
    # print('failed')

    frame_count_load = 0
    frame_count_compute = 0

    print(length)
    print(batch_size)

    for index in tqdm(range((length // batch_size) + 1), position=0, leave=True):

        images = []
        filenames = []

        for index2 in range(batch_size):
            if frame_count_load < length:
                filename = "frame%04d.jpg" % frame_count_load
                path = directory + '/' + imageDir + filename
                images.append(np.array(PIL.Image.open(path)))
                filenames.append(filename)
                frame_count_load += 1
        # filename, image = lst[index]
        # makes sure there is only one face in the image

        outputs = face_recognition.batch_face_locations(images, batch_size=batch_size)

        # Runs batch_size number of times
        for image_idx, output in enumerate(outputs):
            filename = "frame%04d.jpg" % frame_count_compute
            image = images[image_idx]
            frame_count_compute += 1

            if len(output) == 1:
                top, right, bottom, left = output[0]
                cropped_face = image[top - padding:bottom + padding,left - padding:right + padding]

                try:
                  cropped_encoding = face_recognition.face_encodings(cropped_face)[0]
                except IndexError as e:
                  print(filename)
                  continue
                
                # makes sure it is the face that was encoded
                if (face_recognition.compare_faces([face_encoder], cropped_encoding, tolerance=0.7)[0]):
                    frameName = pathOut + filename[0:-4] + "-face-cropped" + ".jpg"
                    cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
                    cv2.imwrite(frameName, cropped_face)     # save cropped as JPEG file

                    if mode == 'test':
                      h, w, c = cropped_face.shape
                      f = open(pathOut + "frame_list.txt", "a")
                      f.write("%s, %d, %d, %d, %d, %d, %d\n" % (filename[5:-4], top, right, bottom, left, h, w))
                      f.close()


                    # print(filename[0:-4] + "-face-cropped" + ".jpg")
                    count += 1

    # print("Cropped " + str(count) + " out of " + (frame_count_compute))

In [ ]:
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

In [ ]:
# image = face_recognition.load_image_file("drive/MyDrive/UMDCP/CMSC/CMSC472/JohnOliver1.jpg")
# oliver_encoding = face_recognition.face_encodings(image)[0]

# image2 = face_recognition.load_image_file("drive/MyDrive/UMDCP/CMSC/CMSC472/JohnOliver1-cropped/frame0103.jpg")
# unknown_encoding = face_recognition.face_encodings(image2)[0]

# results = face_recognition.compare_faces([oliver_encoding], unknown_encoding)
# print(results)

[True]


In [ ]:
# image = face_recognition.load_image_file("drive/MyDrive/UMDCP/CMSC/CMSC472/JohnOliver1.jpg")
# johnny_encoding = face_recognition.face_encodings(image)[0]
# singleFaceExtracter('drive/MyDrive/UMDCP/CMSC/CMSC472/JohnOliver1-cropped/', johnny_encoding)

/content/drive/MyDrive/UMDCP/CMSC/CMSC472/JohnOliver1-cropped-face-crop/
frame0000-face-cropped.jpg
frame0073.jpg
frame0074.jpg
frame0703-face-cropped.jpg
frame1403-face-cropped.jpg
frame1581.jpg
frame1584.jpg


In [ ]:
""" --------------------------- MAIN --------------------------- """
train_file = "Jimmy Fallon.mp4"
train_encoding_frame_number = 45
# test_file = "jimmy.mp4"
# test_encoding_frame_number = 22
input_directory = "drive/MyDrive/UMDCP/CMSC/CMSC472"
output_directory = "drive/MyDrive/UMDCP/CMSC/CMSC472"

# extractImages(train_file, input_directory, output_directory, frame_every=1)
print("Done extracting training frames")

image = face_recognition.load_image_file(output_directory + "/" + train_file[0:-4] + "/frame%04d.jpg" % (train_encoding_frame_number))
jimmy_encoding = face_recognition.face_encodings(image)[0]
singleFaceExtracterWithExtraPixels(train_file[0:-4] + "/", output_directory, jimmy_encoding, padding=0, batch_size=4)
print("Done face cropping training frames")

# extractImages(test_file, input_directory, output_directory)
# print("Done extracting testing frames")

# image = face_recognition.load_image_file(output_directory + "/" + test_file[0:-4] + "/frame%05d.jpg" % (test_encoding_frame_number))
# jimmy_encoding = face_recognition.face_encodings(image)[0]
# singleFaceExtracterWithExtraPixels(test_file[0:-4] + "/", output_directory, jimmy_encoding, padding=0, mode='test', batch_size=4)
# print("Done face cropping testing frames")

print("Output path: " + output_directory)
# print(num_images('drive/MyDrive/UMDCP/CMSC/CMSC472/'))
print("!---------------------------COMPLETE!---------------------------!")

Done extracting training frames


  0%|          | 0/450 [00:00<?, ?it/s]

drive/MyDrive/UMDCP/CMSC/CMSC472/Jimmy Fallon-face-crop/
1798
4


  6%|▌         | 25/450 [00:20<06:04,  1.17it/s]

frame0099.jpg


...

frame1196.jpg
frame1197.jpg
frame1198.jpg


 70%|███████   | 316/450 [04:12<01:49,  1.22it/s]

frame1263.jpg


100%|██████████| 450/450 [05:59<00:00,  1.25it/s]

Done face cropping training frames
Output path: drive/MyDrive/UMDCP/CMSC/CMSC472
!---------------------------COMPLETE!---------------------------!
